#### Preface
В этом практикуме многое позаимствовано у моего коллеги, преподавателя компьютерной лингвистики Миши Нефедова. Спасибо ему.

## Что такое извлечение ключевых слов

Извлечение ключевых слов (keyword extraction) - способ извлечения информации из текста и анализа его тематики. В отличие от автоматического реферирования (саммаризации), мы не создаем полноценный пересказ, а вытаскиваем только ключевые слова (Keywords). Иногда идут чуть дальше и извлекают ключевые словосочетания/n-граммы (Keyphrases). Например, чтобы для текста про "морских коров" не извлекались слова "корова" и "морской", а извлекалось все словосочетание. Это еще актуальнее для более аналитических языков вроде английского (ср. "хотдог" - "hot dog"; вряд ли вы хотите ключевое слово "собака" в тексте про хотдоги).

## Зачем это нужно?

Задача извлечения ключевых слов обычно решается в информационном поиске. Если вы решите писать свой поисковик, или просто индесировать какую-то большую коллекцию текстов, понимать эту тему необходимо. Но и для других задач это может быть полезно. В каком-то смысле, извлечение ключевых слов — простейший способ "тематического моделирования" корпуса. 

## К делу!

### План такой: 
1. Подумать, какие подходы возможны. Реализовать их на случайном примере, оценить результаты глазами

1. Взять готовый набор данных, где ключевые слова уже выделены людьми, и попробовать сравнить с ними (как с эталоном) результат наших методов. Важно помнить, что понятие "эталона" здесь условно. Задача keyword extraction допускает альтернативные решения для одного текста. 

In [ ]:
import os ## работаем с файлами, значит, наверняка понадобится ос
from nltk import word_tokenize ## работаем со словами — значит, токенизатор для этих файлов понадобится
import string ## возьмем оттуда punctuation

## 1 Подходы

### Самый тупой вариант: берем первые и последние слова текста

Давайте возьмем для эксперимента 10 текстов: по 5 новостных и художественных. Скачать можно тут. У меня они лежат в папке 'samples_for_class' рядом с кодом. Положу этот путь в переменную:

In [ ]:
PATH_TO_TEXTS = 'samples_for_class'

Напишем фукнцию, которая берет первые и последние слова.

In [ ]:
def keywords_firstlast (some_text, num_first, num_last):
    tokenized_text = [word for word in word_tokenize (some_text) if word not in string.punctuation]
    if len (tokenized_text) > num_first + num_last:
        return tokenized_text [:num_first] + tokenized_text [num_last*-1:]
    else:
        return tokenized_text

Прогоним по текстам

In [ ]:
file_texts = [] ## в этот пустой список я запишу тексты файлов, чтобы потом пименять к ним разные методы
for some_file in os.listdir (PATH_TO_TEXTS): # в этом цикле я сложу в file_texts тексты файлов, лежащие по адресу в PATH_TO_TEXTS
    if some_file.endswith ('.txt'):
        with open (os.path.join(PATH_TO_TEXTS, some_file),'r') as open_file:
            file_texts.append (open_file.read())

In [ ]:
for text in file_texts:
    print (keywords_firstlast (text, 3,3))

### Второй самый тупой вариант: берем самые частотные слова текста

In [ ]:
from nltk import FreqDist # как вы помните, в нлтк есть счетчик частотностей
# но можно и пользоваться Counter из модуля collections

In [ ]:
def keywords_most_frequent (some_text, num_most_freq):
    tokenized_text = [word for word in word_tokenize (some_text) if word not in string.punctuation+'—»«...']
    return [word_freq_pair[0] for word_freq_pair in FreqDist(tokenized_text).most_common(num_most_freq)]

In [ ]:
for text in file_texts:
    print (keywords_most_frequent (text, 6))

### Третий вариант: берем самые частотные слова текста без стоп-слов

In [ ]:
from nltk.corpus import stopwords 
rus_stops = set(stopwords.words('russian')) ## русские стоп-слова из nltk

In [ ]:
def keywords_most_frequent_with_stop (some_text, num_most_freq, stoplist):
    tokenized_text = [word for word in word_tokenize (some_text) if word not in string.punctuation+'—»«...' and word not in stoplist]
    return [word_freq_pair[0] for word_freq_pair in FreqDist(tokenized_text).most_common(num_most_freq)]

In [ ]:
for text in file_texts:
    print (keywords_most_frequent_with_stop (text, 6, rus_stops))

### Четвертый вариант: берем самые частотные ЛЕММЫ текста без стоп-слов

In [ ]:
from pymystem3 import Mystem ## используем mystem в обертке pymystem

In [ ]:
import re

In [ ]:
moi_analizator = Mystem() ## создаем экземпляр класса "анализатор MyStem"  

In [ ]:
def passed_filter (some_word, stoplist):
    if some_word in string.punctuation +'—»«... ':
        return False
    elif re.search (re.compile('['+string.punctuation+'—»«... \n'+']'), some_word) != None:
        return False
    elif some_word in stoplist:
        return False
    return True

In [ ]:
def keywords_most_frequent_with_stop_and_lemm (some_text, num_most_freq, stoplist):
    lemmatized_text = [word for word in moi_analizator.lemmatize(some_text.lower()) if passed_filter(word, stoplist)]
    return [word_freq_pair[0] for word_freq_pair in FreqDist(lemmatized_text).most_common(num_most_freq)]

In [ ]:
for text in file_texts:
    print (keywords_most_frequent_with_stop_and_lemm (text, 6, rus_stops))

## А какие-нибудь более умные варианты будут?

## 1. Meet TF-IDF ! 

TF IDF — это мера, которая учитывает не только частотность слова в документе (TF, term frequency) — но и то, насколько часто — вернее, насколько редко! — оно встречается во всем корпусе (IDF, inverse document frequency). Формально она считается так:

<img src="https://lh3.googleusercontent.com/proxy/wvgUX_ST82yijsMIG5Sq05e_rY6OrwcUC9wvG4VvElAwdAGUb3oxeofaXucPkZYbyCyW-A6mW2Tboi8OrU96-KR4YSIx6jSDDv12CwRBq26EavdKsXVj">

Суть в том, что таким образом повышаются слова, частотные в данном документе -- и редкие в корпусе в целом. Такие слова получают бонус относительно слов, частотных и в данном документе, и в других. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer ## готовая реализация TF IDF из библиотеки sklearn

In [ ]:
make_tf_idf = TfidfVectorizer (stop_words=rus_stops) # Создаем специальный объект-векторайзер 
#fitted_vectorizer = make_tf_idf.fit(file_texts) 
texts_as_tfidf_vectors=make_tf_idf.fit_transform(file_texts) # Кладем в этот векторайзер наши файлы и просим сделать матрицу TF_IDF

In [ ]:
texts_vectors

In [ ]:
print (texts_vectors.shape) ## посмотрим размерность матрицы

In [ ]:
make_tf_idf.get_feature_names() ## посмотрим 

In [ ]:
## кусочек кода, который берет матрицу TF-IDF и выдает по ней топ-слова для каждого текста

id2word = {i:word for i,word in enumerate(make_tf_idf.get_feature_names())} 
for text_row in range(texts_vectors.shape[0]): 
    row_data = texts_vectors.getrow(text_row) ## берем ряд в нашей матрице -- он соответстует етксты
    words_for_this_text = row_data.toarray().argsort() ## сортируем в нем все слова 
    top_words_for_this_text = words_for_this_text [0,:-6:-1] 
    print ([id2word[w] for w in top_words_for_this_text])

### Естественно, тут можно снова накрутить лемматизацию и стоп-слова получше

## 2. Еще один умный способ: превратить текст в граф (сеть) и искать центральные узлы

Много методов для извлечения ключевых слов основаны на сетевом анализе. Основная идея - каким-то образом перевести текст в граф, а затем посчитать центральности узлов и вывести центральные.  

<img src="https://www.researchgate.net/profile/Mitsuru_Ishizuka/publication/2539694/figure/fig1/AS:669437975883786@1536617859438/A-word-cooccurrence-graph-of-a-set-of-news-articles-The-source-articles-are-a-set-of.png">

Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет основой графа. Ниже приведена НЕ моя реализация сборки такого графа по тексту и примения к нему алгоритма random walk для выделения ключевых слов. Вот описание от автора:

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [ ]:
from itertools import combinations

In [ ]:
### не моя реализация алгоритма

def get_kws(text, top=6, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        
        ### 
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n
    

Испытаем эту реализацию на наших текстах

In [189]:
for text in file_texts:
    print (keywords_most_frequent_with_stop_and_lemm (text, 6, rus_stops))

['россия', 'скворцов', 'инфекция', '10', 'коронавирус', 'коронавирусный']
['денис', 'гайка', 'отвинчивать', 'грузило', 'это', 'понимать']
['год', 'стресс', 'сценарий', 'квартал', 'нефть', 'предприятие']
['тестирование', 'лаборатория', 'тест', 'роспотребнадзор', 'коронавирус', 'биоматериал']
['неделя', 'расход', 'россиянин', 'категория', 'товар', '9']
['собака', 'это', 'очумел', 'палец', 'знать', 'толпа']


## В следующий раз — вторая часть плана

1. Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). В них уже размечены keywords

2. Напишем код для сравнения этих keywords с теми, что выделяют наши функции.